In [1]:
def primal_dual_interior_point(c,Q,A,b,x,pi,z,eta):
    """ 
    Description: 
    this function uses primal-dual interior method to 
    solve convex quadratic constrained optimization
    Input:
    c: coefficient of linear part
    Q: coefficient of quadratic part
    A: coefficient of linear constraint
    b: value of linear constraint
    x, pi, z: column vectors of initial infeasible solution
    eta: damping parameter
    Output: 
    Optimal solution of x,pi,z
    """
    # import library
    import numpy as np
    # set tolerance
    tol=1E-8
    # number of variables
    var_num=len(x)
    # calcualte stopping criteria
    c_1=np.linalg.norm(np.dot(A,x)-b)
    c_2=np.linalg.norm(-np.dot(Q,x)+np.dot(A.T,pi)+z-c)
    c_3=np.dot(x.T,z)
    # initialize counter 
    counter=0
    # print the information of initial infeasible solution 
    print("=====Iteration{}=====".format(counter))
    print('x: {}'.format(x))
    print('pi: {}'.format(pi))
    print('z: {}'.format(z))
    print('tau: {}'.format('-'))
    print('primal problem: {}'.format(np.dot(c.T,x)+0.5*np.dot(x.T,Q).dot(x)))
    print('dual problem: {}'.format(np.dot(b.T,pi)-0.5*np.dot(x.T,Q).dot(x)))
    print('Residual of primal: {}'.format(c_1))
    print('Residual of dual: {}'.format(c_2))
    print('x.T dot x: {}'.format(c_3))
    # check stopping criteria 
    while (c_1 > tol) | (c_2>tol) | (c_3>tol):
        # calculate residules for the infeasible solution
        r_p=np.dot(A,x)-b
        r_d=-np.dot(Q,x)+np.dot(A.T,pi)+z-c
        XZe=np.dot(np.diag(x),np.diag(z).dot(np.ones((var_num))))
        # stack the coefficient matrix
        coef_1=np.vstack((-Q,A,np.diag(z)))
        coef_2=np.vstack((A.T,np.zeros((coef_1.shape[0]-var_num,A.shape[0]))))
        coef_3=np.vstack((np.identity(var_num),np.zeros((A.shape[0],var_num)),np.diag(x)))
        coef=np.hstack((coef_1,coef_2,coef_3))
        # stack the residuals
        r=np.vstack((-r_d.reshape(-1,1),-r_p.reshape(-1,1),-XZe.reshape(-1,1)))
        # solve for affine scaling directions
        d_aff=np.linalg.solve(coef,r)
        # get direction for each part
        d_x_aff=d_aff[:var_num]
        d_pi_aff=d_aff[var_num:coef.shape[0]-var_num]
        d_z_aff=d_aff[-var_num:]
        # find step length of the affine scaling direction
        # create a list to store possible selections
        selections=[1]
        # iterate through possible selections and append result to the list
        for i in range(var_num):
            if d_x_aff[i] < 0:
                selections.append((-x[i]/d_x_aff[i])[0])
            if d_z_aff[i] < 0:
                selections.append((-z[i]/d_z_aff[i])[0])
        # find the minimum value in possible selections 
        selections=np.array(selections)
        alpha_aff=np.min(selections)
        # calculate the duality measure 
        y=np.dot(x.T,z)/var_num
        y_aff=np.dot((x+alpha_aff*d_x_aff.reshape(-1)).T,(z+alpha_aff*d_z_aff.reshape(-1)))/var_num
        # calculate the centering parameter 
        tau=(y_aff/y)**3
        # stack the adjusted residuals
        r_l=-XZe.reshape(-1,1)+np.diag(d_x_aff.reshape(-1)).dot(np.diag(d_z_aff.reshape(-1))).dot(np.ones((var_num,1)))+tau*y*np.ones((var_num,1))
        r_adjust=np.vstack((-r_d.reshape(-1,1),-r_p.reshape(-1,1),r_l))
        # solve for search direction
        d=np.linalg.solve(coef,r_adjust)
        # get direction for each part
        d_x=d[:var_num]
        d_pi=d[var_num:coef.shape[0]-var_num]
        d_z=d[-var_num:]
        # create a list to store possible selections
        selections=[1]
        # iterate through possible selections and append result to the list
        for i in range(var_num):
            if d_x[i] < 0:
                selections.append((-eta*x[i]/d_x[i])[0])
            if d_z[i] < 0:
                selections.append((-eta*z[i]/d_z[i])[0])
        # find the minimum value in possible selections 
        selections=np.array(selections)
        alpha=np.min(selections)
        # calcualte new iterates
        x=x+alpha*d_x.reshape(-1) 
        pi=pi+alpha*d_pi.reshape(-1)
        z=z+alpha*d_z.reshape(-1)
        # calcualte stopping criteria
        c_1=np.linalg.norm(np.dot(A,x)-b)
        c_2=np.linalg.norm(-np.dot(Q,x)+np.dot(A.T,pi)+z-c)
        c_3=np.dot(x.T,z)
        # print the information of each iteration 
        print("=====Iteration{}=====".format(counter+1))
        print('x: {}'.format(x))
        print('pi: {}'.format(pi))
        print('z: {}'.format(z))
        print('tau: {}'.format(tau))
        print('primal problem: {}'.format(np.dot(c.T,x)+0.5*np.dot(x.T,Q).dot(x)))
        print('dual problem: {}'.format(np.dot(b.T,pi)-0.5*np.dot(x.T,Q).dot(x)))
        print('Residual of primal: {}'.format(c_1))
        print('Residual of dual: {}'.format(c_2))
        print('x.T dot x: {}'.format(c_3))
        # update counter
        counter+=1
    return x,pi,z


**Test the function with a quadratic constraint problem:**


In [11]:
import numpy as np
# define the qudratic problem
c=np.array([0,0,0])
Q=np.array([[0.02778,0.00387,0.00021],[0.00387,0.01112,-0.0002],[0.00021,-0.0002,0.00115]])
A=np.array([[0.1073,0.0737,0.0627],[1,1,1]])
b=np.array([0.0650,1])
# define the initial infeasible solution
x=np.array([1,1,1])
pi=np.array([1,1])
z=np.array([1,1,1])

In [12]:
x_optimal,pi_optimal,z_optimal=primal_dual_interior_point(c,Q,A,b,x,pi,z,0.95)

=====Iteration0=====
x: [1 1 1]
pi: [1 1]
z: [1 1 1]
tau: -
primal problem: 0.023905
dual problem: 1.0410949999999999
Residual of primal: 2.0079675520286675
Residual of dual: 3.5772208141656563
x.T dot x: 3
=====Iteration1=====
x: [0.05       0.551804   0.62333086]
pi: [-6.81522183 -0.04482545]
z: [1.01338108 0.7850784  0.70481909]
tau: 0.005894069989513771
primal problem: 0.0019956154242216344
dual problem: -0.48981048232987884
Residual of primal: 0.22603174811563292
Residual of dual: 0.4026785558385087
x.T dot x: 0.9232139443968299
=====Iteration2=====
x: [0.0025     0.50919873 0.61635703]
pi: [-20.16128226   1.02286603]
z: [1.27549507 0.60039681 0.3738499 ]
tau: 0.046299944505073375
primal problem: 0.001602622878477464
dual problem: -0.2892199388894904
Residual of primal: 0.12856590827897474
Residual of dual: 0.2290418700357039
x.T dot x: 0.5393350479333184
=====Iteration3=====
x: [1.25000000e-04 3.44997068e-01 7.11183476e-01]
pi: [-45.90249572   2.90202283]
z: [2.08323231 0.5426497